# Image Augmentation for Pix2Pix

## Imports and installs

In [63]:
import numpy as np
import argparse
import imutils
import cv2
import os, sys, random
from random import randint

In [64]:
!pip install imutils

## Image rotation between -30 and 30 degrees

In [66]:
### Rotate

def rotate_image(mat, angle):
    """
    Rotates an image (angle in degrees) and expands image to avoid cropping
    """

    height, width = mat.shape[:2] # image shape has 3 dimensions
    image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)

    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos = abs(rotation_mat[0,0]) 
    abs_sin = abs(rotation_mat[0,1])

    # find the new width and height bounds
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]

    # rotate image with the new bounds and translated rotation matrix
    rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h))
    return rotated_mat

In [83]:
lt_masks_path = 'C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Final Masked palm/Final Masked palm/'
original_img_pt = 'C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/All palm/All palm/'
new_palm_pt = 'C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Rotate/Palm/'
new_mask_pt = 'C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Rotate/Mask/'
dirs = os.listdir(lt_masks_path)
for file in dirs:
    if os.path.isfile(lt_masks_path+file) and os.path.isfile(original_img_pt+file):
        angle = randint(-30, 30)
        image = cv2.imread(lt_masks_path + file)
        palm_image = cv2.imread(original_img_pt + file)
        rotated_image = rotate_image(image, angle)
        rotated_palm_image = rotate_image(palm_image, angle)
        cv2.imwrite(new_mask_pt + file.split('.jpg')[0]+'_rot.jpg', rotated_image)
        cv2.imwrite(new_palm_pt + file.split('.jpg')[0]+'_rot.jpg', rotated_palm_image)


## Image Flipping ( Random horizontal and vertical flipping)

In [26]:
##### Flipping
def flip(image, flag):
    return cv2.flip(image, flag)

for file in dirs:
    if os.path.isfile(lt_masks_path+file) and os.path.isfile(original_img_pt+file):
        flag = randint(0,1)
        image = cv2.imread(lt_masks_path + file)
        palm_image = cv2.imread(original_img_pt + file)
        rotated_image = flip(image, flag)
        rotated_palm_image = flip(palm_image, flag)
        cv2.imwrite('C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Flip/Mask/' + file.split('.jpg')[0]+'_flip.jpg', rotated_image)
        cv2.imwrite('C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Flip/Palm/' + file.split('.jpg')[0]+'_flip.jpg', rotated_palm_image)


## Random cropping

In [34]:
def random_crop(img, mask, width, height):
    assert img.shape[0] >= height
    assert img.shape[1] >= width
    assert img.shape[0] == mask.shape[0]
    assert img.shape[1] == mask.shape[1]
    x = random.randint(0, img.shape[1] - width)
    y = random.randint(0, img.shape[0] - height)
    img = img[y:y+height, x:x+width]
    mask = mask[y:y+height, x:x+width]
    return img, mask

for file in dirs:
    if os.path.isfile(lt_masks_path+file) and os.path.isfile(original_img_pt+file):
        image = cv2.imread(lt_masks_path + file)
        palm_image = cv2.imread(original_img_pt + file)
        if (image.shape[0] > image.shape[1]):
            rotated_image, rotated_palm_image = random_crop(image, palm_image, image.shape[1], image.shape[1])
        else:
            rotated_image, rotated_palm_image = random_crop(palm_image, image, image.shape[0], image.shape[0])
        cv2.imwrite('C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Crop/Mask/' + file.split('.jpg')[0]+'_crop.jpg', rotated_image)
        cv2.imwrite('C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Crop/Palm/' + file.split('.jpg')[0]+'_crop.jpg', rotated_palm_image)

## Image enhancement

In [51]:
import cv2
import numpy as np

def enhance(img):
    # converting to LAB color space
    lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)

    # Applying CLAHE to L-channel
    # feel free to try different values for the limit and grid size:
    clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L-channel with the a and b channel
    limg = cv2.merge((cl,a,b))

    # Converting image from LAB Color model to BGR color spcae
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    # Stacking the original image with the enhanced image
#     result = np.hstack((img, enhanced_img))
#     cv2.imshow('Result', result)
#     cv2.waitKey(0)

    return enhanced_img



    

## Fading

In [52]:
# img = cv2.imread('C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/All palm/All palm/screen_shot303.png', 1)
def decrease(img):
    imghsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).astype("float32")
    (h, s, v) = cv2.split(imghsv)
    sat = random.uniform(0,2)
#     print(sat)
    s = s*sat
    s = np.clip(s,0,255)
    imghsv = cv2.merge([h,s,v])
    imgrgb = cv2.cvtColor(imghsv.astype("uint8"), cv2.COLOR_HSV2BGR)
#     result = np.hstack((img, imgrgb))
    return imgrgb


## Incorporates all augmented images
### Randomly enhances or degrades the images
 Here, 3 directories were used i.e. images that were flipped, cropped and the original set of images were used

In [85]:
flip_path_mask = 'C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Flip/Mask/'
crop_path_mask = 'C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Crop/Mask/'
palm_path_mask = 'C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Final Masked palm/Final Masked palm/'

flip_path_palm = 'C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Flip/Palm/'
crop_path_palm = 'C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Crop/Palm/'
palm_path      = 'C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/All palm/All palm/'

color_palm = 'C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Color_change/Palm/'
color_mask = 'C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Color_change/Mask/'

dir1 = os.listdir(flip_path_mask)
dir2 = os.listdir(crop_path_mask)
dir3 = os.listdir(palm_path_mask)
print(len(dir1))
print(len(dir2))
print(len(dir3))
for file in dir1:
    
    if os.path.isfile(flip_path_mask+file):
        mask_image = cv2.imread(flip_path_mask + file)
        palm_image = cv2.imread(flip_path_palm + file)
        s = random.randint(0,1)
        
        if s == 0:
            new_image = decrease(palm_image)
        else:
            new_image = enhance(palm_image)
        cv2.imwrite('C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Color_change/Palm/' + file.split('.jpg')[0]+'_colr.jpg', new_image)
        cv2.imwrite('C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Color_change/Mask/' + file.split('.jpg')[0]+'_colr.jpg', mask_image)
        
    
for file in dir2:
    
    if os.path.isfile(crop_path_mask+file):
        mask_image = cv2.imread(crop_path_mask + file)
        palm_image = cv2.imread(crop_path_palm + file)
        s = random.randint(0,1)
        if s == 0:
            new_image = decrease(palm_image)
        else:
            new_image = enhance(palm_image)
        cv2.imwrite('C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Color_change/Palm/' + file.split('.jpg')[0]+'_colr.jpg', new_image)
        cv2.imwrite('C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Color_change/Mask/' + file.split('.jpg')[0]+'_colr.jpg', mask_image)
        
for file in dir3:
    print(len(dir3))
    if os.path.isfile(palm_path_mask+file):
        mask_image = cv2.imread(palm_path_mask + file)
        palm_image = cv2.imread(palm_path + file)
        s = random.randint(0,1)
        if s == 0:
            new_image = decrease(palm_image)
        else:
            new_image = enhance(palm_image)
        cv2.imwrite('C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Color_change/Palm/' + file.split('.jpg')[0]+'_colr.jpg', new_image)
        cv2.imwrite('C:/Users/nramesh/Documents/Satellite/Images/Pix2Pix/Image_Augmentation/Color_change/Mask/' + file.split('.jpg')[0]+'_colr.jpg', mask_image)        


251
251
251
